In [ ]:
# 필요한 패키지 불러오지
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_1 = pd.read_csv('/content/book.csv', encoding='euc-kr')

# df_1.head()

In [ ]:
df_1.tail()

In [ ]:
df_1 = df_1.dropna(subset=['Title'])
df_1 = df_1[df_1['Title'].str.strip() != '']

df_2 = df_1.dropna(subset=['Publisher'])
df_2 = df_2[df_2['Publisher'].str.strip() != '']


In [ ]:
print(df['Price'].dtypes)

float64


In [ ]:
# 한국어 타이틀 레코드만 가져옴
df = df_2[df_1['Title'].str.contains('[가-힣]', regex=True)]
# 한국어 [가-힣]
# 영어[A-Za-z]
# 일본어 [ぁ-んァ-ン]


<ipython-input-5-2359c6cfc5ca>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df_2[df_1['Title'].str.contains('[가-힣]', regex=True)]


In [ ]:
# price  object => 수치형
df['Price'] = pd.to_numeric(df['Price'].str.replace('[^\d.]', ''), errors='coerce')

# 'Pdate' 컬럼을 날짜 타입으로 변환 (한국어 날짜 형식에 맞춤)
df['Pdate'] = pd.to_datetime(df['Pdate'], format='%Y년 %m월 %d일', errors='coerce')


In [ ]:
# 클러스터링을 위한 패키지
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [ ]:
# 벡터화
df['t_p'] = df['Publisher'] + "_" + df['Title']

tfidf_vectorizer = TfidfVectorizer(max_features=100) # 전체 단어 집합에서 TF-IDF 값이 가장 높은 상위 n개의 단어만을 선택하여 특성 벡터를 생성
X_tfidf = tfidf_vectorizer.fit_transform(df['t_p'])

<ipython-input-8-c3b4e2e40ed2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t_p'] = df['Publisher'] + "_" + df['Title']


In [ ]:
# 클러스터링 수행
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(X_tfidf)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10, random_state=42)

In [ ]:
# 결과 할당
df['Cluster']=kmeans.labels_

<ipython-input-11-60510dac6cfb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cluster']=kmeans.labels_


In [ ]:
# 클러스터 별로 데이터 확인
for cluster in range(10): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df[df['Cluster'] == cluster]['Title'].head(), '\n') # 각 클러스터에 속한 책 제목 예시 출력

Cluster 0:
12         강철왕국 프로이센
25           이화림 회고록
66      쓰레기에 관한 모든 것
90    고전 기하학과 현대 기하학
91       실전형 MES 방법론
Name: Title, dtype: object 

Cluster 1:
476         런웨이의 연인 1
484        드러그 & 드롭 1
1761    바람의 검심 완전판 10
1762    바람의 검심 완전판 11
1763    바람의 검심 완전판 12
Name: Title, dtype: object 

Cluster 2:
1041    [대여] 마이페이스 상사와 위장결혼 (개정판) 084화
1048        [대여] [연재] 바람의 나라 SE 4부 65화
1053              [대여] [컬러 연재] 도화선 18화
1055    [대여] 레이디로즈는 평민이 되고 싶어 03권 (완결)
1058         [대여] [컬러 연재] 로스트 프린세스 63화
Name: Title, dtype: object 

Cluster 3:
24       로마인 이야기 1-15권 세트
351      맛있는 햄버거의 무서운 이야기
575        진순신의 삼국지 이야기 1
576        진순신의 삼국지 이야기 2
882    말랑하고 쫀득한 세계사 이야기 1
Name: Title, dtype: object 

Cluster 4:
1692     수학도둑 1
1851    수학도둑 39
1867    수학도둑 50
1868    수학도둑 41
1904    수학도둑 36
Name: Title, dtype: object 

Cluster 5:
922       여자의 시간
994       여자의 시간
1143    여고생의 치맛단
1203       마음 깨침
1319    여고생의 치맛단
Name: Title, dtype: object 

Cluster 6:
1679       Brown of Calaveras (영어로 세계문학읽기 96

In [ ]:
# 클러스터 결과 시각화
# T-SNE  차원 축소 방법 사용
# 고차원의 TF-IDF 벡터를 2차원으로 매핑한 후 클러스터링 결과를 시각화
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_tfidf.toarray())
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=kmeans.labels_)
plt.show()

Word2Vec 모델

In [ ]:
df['t_p'] = df['Title']
# df['t_p'] = df['Publisher'] + "_" + df['Title']

<ipython-input-20-3113282234a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t_p'] = df['Title']


In [ ]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

# 텍스트 전처리

# 문서를 단어 리스트로 변환
documents = [text.split() for text in df['t_p']]


In [ ]:
documents[0]

['강철왕국', '프로이센']

In [ ]:
# Word2Vec: 단어의 벡터 변환 모델 중 하나, 비슷한 의미를 가진 단어들이 벡터 공간에서 서로 가까이 위치하도록 학습
model = Word2Vec(sentences=documents, vector_size=100, window=5, min_count=1, workers=4)

# sentences=documents: 학습에 사용될 데이터(문서) 리스트 포맷(각 문서 내 단어들을 리스트)
# vector_size=100: 생성될 단어 벡터 차원 수
# window=5: 단어 예측을 위해 고려하는 주변 단어의 범위. 5:타겟 단어의 앞뒤로 5개의 단어를 컨텍스트로 사용
# min_count=1: 학습에 포함될 최소 단어 빈도수
# workers=4: 스레드의 수(병렬 처리)


#  문서 벡터 생성(<- 단어 벡터 평균)
def document_vector(word2vec_model, doc):
    vector_size = word2vec_model.vector_size   # 단어 벡터 차원 수

    doc_vector = np.zeros(vector_size) # 문서 벡터(초기값 0) 문서 내 각 단어의 벡터를 더함
    num_words = 0
    for word in doc:
        try:
            doc_vector += word2vec_model.wv[word]
            num_words += 1
        except KeyError: # 문서 내 단어의 수
            continue
    if num_words == 0:
        return np.zeros(vector_size)
    else:
        return doc_vector / num_words  # 각 단어 벡터를 더해서 구해진 doc_vector를 num_words로 나누어 평균 계산 (단어 벡터 평균)
                                       # 클러스터링의 input 으로 사용

# 각 문서 벡터 생성
doc_vectors = np.array([document_vector(model, doc) for doc in documents])


In [ ]:
vector = model.wv['강철왕국']
vector

array([-1.7443757e-03,  7.6518459e-03,  6.1961268e-03,  8.6477762e-03,
        2.0360427e-03, -1.6407197e-03,  6.1720056e-03,  1.0568601e-03,
        9.2386699e-04, -5.4063117e-03,  8.4783528e-03,  5.6946860e-03,
       -3.5455541e-03,  9.6020871e-04,  5.0485968e-03, -9.8427869e-03,
        3.9142175e-03,  5.3081615e-03, -9.3773589e-05, -4.2427210e-03,
        9.2965122e-03,  5.9067504e-03,  9.8650772e-03, -8.3465502e-03,
        5.9848502e-03, -7.9352045e-03,  3.8676881e-03, -9.1035636e-03,
       -7.5577712e-03,  4.0470224e-04, -3.4220216e-03,  5.7825893e-03,
        9.4705420e-03,  2.9901199e-03,  7.3269992e-03,  5.4056444e-03,
       -8.2684141e-03,  1.4069189e-03, -2.5677308e-03, -9.9521279e-03,
        9.0282839e-03, -3.5276860e-03, -6.0148751e-03, -6.1616250e-03,
        8.6651342e-03, -7.6915142e-03,  9.0629951e-04, -8.9489883e-03,
        2.3820850e-03, -1.0413794e-03, -9.8278858e-03,  8.2722371e-03,
       -5.6711566e-03,  8.2409205e-03,  6.2419251e-03, -7.7138552e-03,
      

In [ ]:
# K-평균 클러스터링 모델 생성 및 학습
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(doc_vectors)

# 클러스터 할당 결과
df['Cluster'] = kmeans.labels_


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-24-63018587f320>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cluster'] = kmeans.labels_


In [ ]:
# 클러스터 별로 데이터 확인
for cluster in range(5): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df[df['Cluster'] == cluster]['Title'].head(), '\n') # 각 클러스터에 속한 책 제목 예시 출력

Cluster 0:
12         강철왕국 프로이센
25           이화림 회고록
66      쓰레기에 관한 모든 것
90    고전 기하학과 현대 기하학
91       실전형 MES 방법론
Name: Title, dtype: object 

Cluster 1:
458       SAS 서바이벌 가이드
1711         쿠키런 한자런 7
1733      쿠키런 개그 과학 상식
1743         쿠키런 한자런 8
1837    쿠키런 신대륙에서 찾아라!
Name: Title, dtype: object 

Cluster 2:
218     주류 창업을 위한 주류면허 길라잡이
344         생각하는 아이 기다리는 엄마
351        맛있는 햄버거의 무서운 이야기
369      20대를 위한 취업에 성공하는 법
370    내 인생을 풍요롭게 만드는 한편의 시
Name: Title, dtype: object 

Cluster 3:
1719     쿠키런 서바이벌 대작전 7
1738     쿠키런 서바이벌 대작전 2
1739     쿠키런 서바이벌 대작전 1
1740     쿠키런 서바이벌 대작전 3
1768    쿠키런 서바이벌 대작전 16
Name: Title, dtype: object 

Cluster 4:
24     로마인 이야기 1-15권 세트
93           건축견적이야기 2 
94           건축견적이야기 3 
192         건축견적이야기 세트 
357           예수님의 사람 2
Name: Title, dtype: object 



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 텍스트 토큰화 및 시퀀스 변환
tokenizer = Tokenizer()
# Tokenizer 객체를 사용하여 텍스트를 토큰화, 각 토큰(단어)에 고유한 정수 인덱스 할당
tokenizer.fit_on_texts(df['Title'])
X_seq = tokenizer.texts_to_sequences(df['Title'])

In [ ]:
################# FastText  #################
from gensim.models import FastText

# 예제 문장들
sentences = [['cat', 'say', 'meow'], ['dog', 'say', 'woof']]

# FastText 모델 훈련
model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 'cat' 단어의 벡터 조회
print(model.wv['cat'])

[-3.08066304e-03 -7.90226390e-04 -2.73370859e-03  1.61904667e-03
 -1.66713522e-04  6.28919399e-04  9.15853481e-04  3.49709508e-03
  1.62937783e-03  4.91646770e-03  2.77808547e-04 -1.39591098e-03
 -3.55925737e-03  1.01852708e-03  5.35819272e-04  1.28516625e-03
  4.77675669e-04  1.46795274e-03 -4.46858292e-04 -1.06995976e-04
 -2.04688148e-03  2.61852262e-03 -7.45429657e-04  2.08033761e-03
 -1.13760144e-03 -3.43189389e-03 -3.78587609e-03 -7.24313839e-04
 -1.30828656e-03  1.19137007e-03  7.66331214e-04  4.83161490e-03
  9.78983124e-04  3.17783700e-03  2.34751962e-03 -2.66464660e-03
 -1.32662349e-03 -4.01243247e-04  3.82719038e-04 -2.54542427e-03
 -2.49923527e-04 -4.83285869e-03 -4.70269052e-03  2.44425796e-03
 -4.80298093e-03 -3.20287934e-03  2.02403011e-04  2.88115279e-03
 -2.96206702e-03 -1.78865588e-03 -4.56541049e-04 -2.65269354e-03
  3.16352444e-03 -9.34481970e-04 -3.65442189e-04  1.79229118e-03
 -1.84320379e-03  1.56961230e-03  2.01237039e-03 -4.33722837e-03
  1.29108585e-03  4.41135

In [ ]:
################# FastText  #################
from gensim.models import FastText
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# FastText 모델 학습 (로컬 데이터에서)
# sentences = [title.split() for title in df['Title']]
sentences = [title.split() for title in df['Title']]
fasttext_model = FastText(X_seq, vector_size=64, window=5, min_count=1)

# 책 제목 -> 벡터화
def vectorize_w_fasttext(text):
    words = text.split()
    word_vectors = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(fasttext_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 제목 벡터화
title_vectors = np.array([vectorize_w_fasttext(title) for title in df['Title']])



In [ ]:
# 유사한 단어
model.wv.most_similar('영어')

[('say', 0.20480839908123016),
 ('dog', -0.023522047325968742),
 ('cat', -0.1022270917892456),
 ('meow', -0.15924866497516632),
 ('woof', -0.2406514286994934)]

In [ ]:
# 두 단어간 유사도 확인
model.wv.similarity('수학', '영어')

0.26143593

In [ ]:
# K-평균 클러스터링 모델 생성 및 학습
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(title_vectors)

# 클러스터 할당 결과
df['Cluster'] = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-30-b863022140d7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cluster'] = kmeans.labels_


In [ ]:
# 클러스터 별로 데이터 확인
for cluster in range(10): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df[df['Cluster'] == cluster]['Title'].head(), '\n') # 각 클러스터에 속한 책 제목 예시 출력


Cluster 0:
12           강철왕국 프로이센
24    로마인 이야기 1-15권 세트
25             이화림 회고록
66        쓰레기에 관한 모든 것
90      고전 기하학과 현대 기하학
Name: Title, dtype: object 

Cluster 1:
357         예수님의 사람 2
409    묻고 답하는 한국사카페 1
422           밤의 양들 2
457            이리 와 2
461          엉터리 릴리 4
Name: Title, dtype: object 

Cluster 2:
541    RURE 루어 23
545    RURE 루어 15
547         마기 27
550         마기 27
551    RURE 루어 17
Name: Title, dtype: object 

Cluster 3:
1527          식객 18
1802           궁 26
1882    위기탈출 넘버원 10
1902      드래곤빌리지 26
1903      드래곤빌리지 20
Name: Title, dtype: object 

Cluster 4:
1753    드래곤빌리지 학습도감 11 : 애견천국
1756    드래곤빌리지 학습도감 12 : 딱정포스
1763            바람의 검심 완전판 12
1765            바람의 검심 완전판 13
1768          쿠키런 서바이벌 대작전 16
Name: Title, dtype: object 

Cluster 5:
371           나, 지금 죽어도 좋아 2
372          빌 클린턴의 마이 라이프 2
408    요리스타 청 1 천재 요리 소녀의 등장
520          반쪽 달이 떠오르는 하늘 1
536                  여왕의 꽃 9
Name: Title, dtype: object 

Cluster 6:
869     2020 건축시공
941     2020 건축시공
118